<a href="https://colab.research.google.com/github/lucasbenazzi/FCC-MachineLearningExercises/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  #!pip install tf-nightly
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.7.0


In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2022-01-26 13:39:03--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv.1’

train-data.tsv.1    100%[===================>] 349.84K  --.-KB/s    in 0.003s  

2022-01-26 13:39:03 (105 MB/s) - ‘train-data.tsv.1’ saved [358233/358233]

--2022-01-26 13:39:04--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv.1’

valid-data.tsv.1    100%[=========

In [ ]:
# extra imports
from keras.preprocessing import sequence

In [ ]:
#  create dataframes from files
df_train = pd.read_csv(train_file_path, sep='\t', header = None, names = ['status','text'])
df_test = pd.read_csv(test_file_path, sep='\t', header = None, names = ['status','text'])

In [ ]:
# we need to define a set lenght for each input

# create a series with the lenght of words on each text
# this steps also normalizes the words the same way will normalize later
aux_df = df_train['text'].copy().\
str.replace('[^a-zA-Z]', ' ').\
str.replace('\s+', ' ', regex=True).\
str.lower().\
str.split().\
str.len()

print(aux_df.describe())

# we can see that most texts have fewer then 23 words
MAX_LENGTH = int(aux_df.quantile(0.75))

count    4179.000000
mean       15.708064
std        11.519769
min         0.000000
25%         7.000000
50%        12.000000
75%        23.000000
max       190.000000
Name: text, dtype: float64


In [ ]:
# since we need numbers as inputs, create a word_map variable to map every word in the training set to a number
def create_word_map(dataframe):
  df = dataframe.copy()
  
  # only leave alphabetical values, and make all words lower case
  df['text'] = df['text'].str.replace('[^a-zA-Z]', ' ').str.replace('\s+', ' ', regex=True).str.lower()

  # create a dataframe with the value count of each word, so words that appear more often have a lower number attributed to it
  word_map  = df['text'].str.split(expand=True).stack().value_counts().reset_index().rename(columns = {'index':'word', 0:'count'})

  # disregard words that appear only once
  word_map =  word_map.loc[word_map['count'] > 1]

  # create a new column with index value + 1, so the most common word is 1 instead of 0
  word_map['id'] = word_map.index + 1

  return word_map[['word','id']].set_index('word')

In [ ]:
# returns a "values" and a "labels" variables based on the "word_map" variable
def prepare_numpy_array(dataframe, word_map):
  df = dataframe.copy()

  # convert 'status' column to numbers
  df['status'] = df['status'].replace({
          'ham':0,'spam':1})
  
  # only leave alphabetical values, and make all words lower case
  df['text'] = df['text'].str.replace('[^a-zA-Z]', ' ').str.replace('\s+', ' ', regex=True).str.lower()

  # create a numby array for both "text" column and "status"
  text_array = df['text'].to_numpy()

  # create a new list where each word is converted to it's corresponding number
  # this method ussed instead of the "pandas.apply" function to save processing time
  value_dataset = []
  for index in range(text_array.shape[0]):
    text_list = text_array[index].split()
    number_list = []

    for word in text_list:
      # if the word isn't present in the word map variable, disregard it
      if word in word_map.index:
        number_list.append(word_map.loc[word][0])

    value_dataset.append(number_list)

  # add leading zeros or delete extra words in order to standardize the length according to the MAX_LENGTH variable
  value_dataset = sequence.pad_sequences(value_dataset, MAX_LENGTH)

  return np.array(value_dataset), np.array(df['status'])

In [ ]:
word_map = create_word_map(df_train)

train_data, train_labels = prepare_numpy_array(df_train, word_map)

In [ ]:
# add 1 to the vocabulary size to account empty (zero value) elements
VOCAB_SIZE = word_map.shape[0] + 1

# create, compile and train the model
# for this classification problem will first ad a word embedding layer followed by a lstm layer
# since it's a binary classification problem, the last layer will have one neuron and have a sigmoid activation function
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=['acc']
    )

# preparting the test dataframe to use as validation
test_data, test_labels = prepare_numpy_array(df_test, word_map)

model.fit(train_data, train_labels, epochs=10, validation_data = (test_data, test_labels))

Epoch 1/10
131/131 [==============================] - 5s 19ms/step - loss: 0.2553 - acc: 0.9179 - val_loss: 0.0916 - val_acc: 0.9792
Epoch 2/10
131/131 [==============================] - 2s 15ms/step - loss: 0.0528 - acc: 0.9885 - val_loss: 0.0429 - val_acc: 0.9871
Epoch 3/10
131/131 [==============================] - 2s 16ms/step - loss: 0.0259 - acc: 0.9935 - val_loss: 0.0455 - val_acc: 0.9835
Epoch 4/10
131/131 [==============================] - 2s 16ms/step - loss: 0.0160 - acc: 0.9962 - val_loss: 0.0397 - val_acc: 0.9892
Epoch 5/10
131/131 [==============================] - 2s 15ms/step - loss: 0.0090 - acc: 0.9981 - val_loss: 0.0404 - val_acc: 0.9885
Epoch 6/10
131/131 [==============================] - 2s 16ms/step - loss: 0.0057 - acc: 0.9986 - val_loss: 0.0376 - val_acc: 0.9899
Epoch 7/10
131/131 [==============================] - 2s 15ms/step - loss: 0.0047 - acc: 0.9988 - val_loss: 0.0367 - val_acc: 0.9914
Epoch 8/10
131/131 [==============================] - 2s 16ms/step - 

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  # transform the string into an array with the same rules applied to the training data

  # convert text into a dataframe, to take advantage of the already created functions "prepare_numpy_array"
  pred_text_df = pd.DataFrame(data = [['ham', pred_text]], columns = ['status','text'])

  pred_text, _ = prepare_numpy_array(pred_text_df, word_map)

  # make the prediction
  prediction = model.predict(pred_text)[0][0]
  
  if prediction >= 0.5:
    prediction_status = 'spam'
  else:
    prediction_status = 'ham'

  return [prediction, prediction_status]

pred_text = "how are you doing today?"
prediction = predict_message(pred_text)
print(prediction)

[7.8286655e-05, 'ham']


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


You passed the challenge. Great job!


##Sources

1. "pandas.read_csv" pandas, https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html
2. "f.keras.preprocessing.sequence.pad_sequences" TensorFlow, https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences
3. "How to Make Predictions with Long Short-Term Memory Models in Keras" Machine Learning Mastery, https://machinelearningmastery.com/make-predictions-long-short-term-memory-models-keras/
4. "6 Practices to enhance the performance of a Text Classification Model" Analytics Vidhya, https://www.analyticsvidhya.com/blog/2015/10/6-practices-enhance-performance-text-classification-model/


##Extra links

This project was also based in part on the Google colab file from FreeCodeCamp "Machine Learning with Python" Course.

https://colab.research.google.com/drive/1ysEKrw_LE2jMndo1snrZUh5w87LQsCxk#forceEdit=true&sandboxMode=true&scrollTo=OWGGcBIpjrMu

###The following Stackoverflow links where used to help on this project

1. https://stackoverflow.com/questions/9652832/how-to-load-a-tsv-file-into-a-pandas-dataframe
2. https://stackoverflow.com/questions/46241120/how-to-remove-non-alpha-numeric-characters-from-strings-within-a-dataframe-colum
3. https://stackoverflow.com/questions/43071415/remove-multiple-blanks-in-dataframe
4. https://stackoverflow.com/questions/46786211/counting-the-frequency-of-words-in-a-pandas-data-frame